In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ipywidgets as widgets
%matplotlib widget

<h1>RED NEURONAL</h1>
<h2>Aplicación de gradiente descendente en una red de una única capa oculta.</h2>
<ul>
    <li>Diferencias en la curva de error con diferentes tasas de aprendizaje.</li>
    <li>Diferencias en la curva de error con diferente número de neuronas en la capa oculta.</li>
    <li>Diferencias en la curva de error con inicialización de las matrices $\Theta$ en ceros.</li>
</ul>

In [ ]:
x1 = np.random.randint(2, size = 40).reshape(-1,1)
x2 = np.random.randint(2, size = 40).reshape(-1,1)

Y = np.logical_not(np.logical_xor(x1,x2))

In [ ]:
X_noBias =  np.hstack((x1,x2))

In [ ]:
X = np.hstack((np.ones(x1.shape[0]).reshape(-1,1), X_noBias))
m = X.shape[0]

In [ ]:
#numero neuronas en las 2da capa sin contar el bias unit
neurons_sel = widgets.Dropdown(
    options = [1,2,3,4,5],
    value = 2,
    description ='Neurons:',
    layout = {'width': 'max-content'}
)

In [ ]:
def costFunction(X, Y, theta1, theta2, l=0):
    a1 = X

    z2 = np.dot(a1, theta1)
    a2 = 1 / (1 + np.exp(-z2))
    a2 = np.insert(a2, 0, 1, axis=1)

    z3 = np.dot(a2, theta2)
    a3 = 1 / (1 + np.exp(-z3))

    cost = np.dot(Y.T, np.log(a3)) + np.dot((1 - Y).T, np.log(1 - a3))

    reg = l / (2 * m) * (np.sum(np.square(theta1[1:,:])) + np.sum(np.square(theta2[1:,:])))

    J = -1 / m * cost + reg

    return J

In [ ]:
def backpropagation(X, Y, theta1, theta2, l=0): 

    Delta1 = np.zeros(theta1.shape)
    Delta2 = np.zeros(theta2.shape)

    for i in range(0,m):
        a1 = X[i,:].reshape(1,-1)

        z2 = np.dot(a1, theta1)

        a2 = 1 / (1 + np.exp(-z2))

        a2 = np.insert(a2, 0, 1, axis=1)

        z3 = np.dot(a2, theta2)

        a3 = 1 / (1 + np.exp(-z3))

        d3 = a3 - Y[i]

        d2 = (np.dot(theta2[1:,:], d3)) * a2[:,1:].T * (1 - a2[:,1:].T)

        Delta2 += np.dot(a2.T, d3)
        Delta1 += np.dot(a1.T, d2.T)

    theta1_grad = 1 / m * Delta1
    theta2_grad = 1 / m * Delta2

    theta1_grad[:,1:] += (l / m * theta1[:,1:]);
    theta2_grad[:,1:] += (l / m * theta2[:,1:]);
    
    return theta1_grad, theta2_grad

In [ ]:
a_slider = widgets.FloatSlider(min = 0, max = 1, step = 0.001, value = 0.5, description = r'$\alpha$')

In [ ]:
def J_function(X, Y, theta1, theta2, alpha):
    J = np.array([])
    
    for j in range(0,1000):
        J = np.append(J, costFunction(X, Y, theta1, theta2))
        theta1_grad, theta2_grad = backpropagation(X, Y, theta1, theta2)
        theta1 = theta1 - alpha * theta1_grad
        theta2 = theta2 - alpha * theta2_grad
    
    return J, theta1, theta2

In [ ]:
plot_J = widgets.Output()

with plot_J:
    fig_J, ax_J = plt.subplots(figsize = (6,4),tight_layout = True)
    fig_J.suptitle(r'Variación función costo $J(\theta)$ con gradiente descendente')
    
    theta1 = np.random.rand(X.shape[1],neurons_sel.value)
    theta2 = np.random.rand(neurons_sel.value+1,1)
    J, theta1, theta2 = J_function(X,Y,theta1,theta2,a_slider.value)
        
    lineJ, = ax_J.plot(J)
    ax_J.grid(True);

In [ ]:
def update_plotJ(change):
    theta1 = np.random.rand(X.shape[1], neurons_sel.value)
    theta2 = np.random.rand(neurons_sel.value+1,1)
    J, theta1, theta2 = J_function(X, Y, theta1, theta2, a_slider.value)
    lineJ.set_ydata(J)

def updateWrong_plotJ(change):
    theta1 = np.zeros((X.shape[1], neurons_sel.value))
    theta2 = np.zeros((neurons_sel.value+1,1))
    J, theta1, theta2 = J_function(X, Y, theta1, theta2, a_slider.value)
    lineJ.set_ydata(J)

    
b1 = widgets.Button(description = 'Nuevo inicio', button_style = 'warning')
b1.on_click(update_plotJ)

b2 = widgets.Button(description = 'Inicio en 0', button_style = 'danger')
b2.on_click(updateWrong_plotJ)

layout = widgets.Layout(align_items = 'center')
    
a_slider.observe(update_plotJ,'value')
neurons_sel.observe(update_plotJ,'value')

widgets.VBox([plot_J, a_slider, neurons_sel, b2, b1], layout = layout)

<div class="alert alert-block alert-danger">
<b>Conclusiones:</b> 
    <ol>
        <li>No es recomendable inicializar todos los valores de las matrices $\Theta$ 
            como 0. Ya que al realizar la propagación hacia adelante y hacia atrás, 
            cada nodo realiza el mismo cálculo y cada valor de la matriz $\Theta$ 
            será actualizado al mismo valor.  </li>
         <li>Una red con sus matrices $\Theta$ puede no aprender nada, o realizarlo 
             muy lentamente. </li>
        <li>Los valores de las matrices $\Theta$ deben ser inicializados de manera 
            aleatoria.</li>
    </ol>
</div>

<div  class="alert alert-block alert-warning">
<b>Conclusiones:</b> 
    <ol>
        <li>Debido a la aleatoriedad en la selección de los valores de las 
            matrices $\Theta$ y a que la función $J(\Theta)$ no es convexa, es 
            muy probable obtener mínimos locales al utilizar gradiente descendente 
            y no el mínimo global de la función costo.</li>
    </ol>
</div>

<h2>Mejora del resultado obtenido al aplicar gradiente descendente.</h2>
<ul>
    <li>Realización de múltiples entrenamientos con distinta inicialización 
    de los parámetros $\Theta$.</li>
</ul>

In [ ]:
a_slider_h = widgets.FloatSlider(min = 0, max = 1, step = 0.001, value = 0.5, description = r'$\alpha$')

neurons_h = widgets.Dropdown(
    options = [1,2,3,4,5,6],
    value = 2,
    description = 'Neurons:',
    layout = {'width': 'max-content'}
)

#Numero de veces que se inicializan los theta para buscar el mejor caso
tests = widgets.Dropdown(
    options = [i for i in range(1,11)],
    value = 10,
    description = 'Trainings:',
    layout = {'width': 'max-content'}
)


In [ ]:
def find_bestJ(neurons, alpha):
    best_J = 100
    Jvalues = np.array([])
    
    for i in range(tests.value):
        theta1 = np.random.rand(X.shape[1],neurons)
        theta2 = np.random.rand(neurons+1, 1)
        J, theta1,theta2 = J_function(X, Y, theta1, theta2, alpha)
        Jvalues = np.append(Jvalues, J[-1])
        if J[-1] < best_J:
            best_J = J[-1]
            best_t1 = theta1
            best_t2 = theta2
    
    return best_J, best_t1, best_t2, Jvalues

In [ ]:
best_J, best_t1, best_t2, Jvalues = find_bestJ(neurons_h.value, a_slider_h.value)
out_J = widgets.HTMLMath(value = fr'El valor de $J(\theta)$ es: {best_J:.5f}')
out_Jextra = widgets.HTMLMath(value = fr'con una media de {Jvalues.mean():.5f} y varianza {Jvalues.var():.5f}')


In [ ]:
def predictor(theta1, theta2):
    a1 = X
    
    z2 = np.dot(a1,theta1)
    a2 = 1/(1 + np.exp(-z2))
    a2 = np.insert(a2, 0, 1, axis=1)

    z3 = np.dot(a2,theta2)
    a3 = 1/(1+np.exp(-z3))
    
    h = (a3 > 0.5)*1
    
    return h

In [ ]:
plot_h = widgets.Output()

with plot_h:
    fig_h, ax_h = plt.subplots(figsize = (8,4),tight_layout = True)
    fig_h.suptitle(r'Hipótesis vs Valores de salida reales')
    h = predictor(best_t1,best_t2)
    lineh, = ax_h.plot(np.arange(0,len(Y)), h,'rx', label='Valor predicho')
    ax_h.scatter(np.arange(0,len(Y)),Y,marker='o',edgecolors = 'b',facecolors = 'none', s=80, label='Valor real')
    ax_h.grid(True)
    ax_h.legend();


In [ ]:
def update_ploth(change):
    best_J, best_t1, best_t2, Jvalues = find_bestJ(neurons_h.value, a_slider_h.value)
    h = predictor(best_t1, best_t2)
    lineh.set_ydata(h)
    out_J.value = fr'El valor de $J(\theta)$ es: {best_J:.5f}'
    out_Jextra.value = fr'con una media de {Jvalues.mean():.5f} y varianza {Jvalues.var():.5f}'
    
b1_h = widgets.Button(description = 'Nuevo inicio',button_style = 'success')
b1_h.on_click(update_ploth)

layout = widgets.Layout(align_items = 'center')
    
a_slider_h.observe(update_ploth,'value')
neurons_h.observe(update_ploth,'value')
tests.observe(update_ploth,'value')

widgets.VBox([plot_h, a_slider_h, neurons_h,tests,out_J, out_Jextra, b1_h],layout = layout)


<div  class="alert alert-block alert-success">
<b>Conclusiones:</b> 
    <ol>
        <li>Una forma de mejorar la obtención de los valores  
            de las matrices $\Theta$ es realizando un mayor número de 
            entrenamientos a partir de distintas matrices $\Theta$ de
            valores aleatorios y luego seleccionar aquellas que permitan 
            minimizar la función costo $J(\Theta)$.</li>
    </ol>
</div>

<ul>
     <li>Variación del valor de la función costo luego de 1000
      épocas de entrenamiento, para redes con distinto
      número de neuronas en la caa .</li>
</ul>

In [ ]:
allJ = np.zeros((10,6))

for neurons in range(1,7):
     best_J, best_t1, best_t2, Jvalues = find_bestJ(neurons, 0.5)
     allJ[:,neurons-1] = Jvalues
 
df_allJ = pd.DataFrame(allJ)
df_allJ.columns = ['1','2','3','4','5','6']

sns.catplot(data = df_allJ, kind = "box").set(xlabel = 'N° de neuronas', \
                                            ylabel = f'$J(\Theta$)')
sns.set_style("whitegrid")
plt.title('Variación de la función costo')
plt.tight_layout()


<div  class="alert alert-block alert-success">
<b>Conclusiones:</b> 
    <ol>
        <li>Al aumentar el número de neuronas en la capa oculta el 
            resultado tiende a mejorar, aunque esto implica un mayor 
            costo computacional.</li>
    </ol>
</div>

<h2>Implementación con librerias especializadas</h2>

<ol>
    <li>Se utiliza el paquete <strong>scikit-learn</strong>
    </li>
</ol>

In [ ]:
from sklearn.neural_network import MLPClassifier 

<ol start="2">
    <li>Se crea una instancia de la clase <strong>MLPClassifier</strong>, la cual 
        tiene muchos parámetros opcionales:
        <ul>
          <li><strong>hidden_layer_sizes</strong> tupla conformada por tantos 
              elementos como capas ocultas posea la red, el elemento i representa 
              el número de neuronas de la capa oculta i (sin considerar los nodos
              de tipo bias.</li>
          <li><strong>activation</strong> función de activación utilizada: 'identity', 
              'logistic', 'tanh', 'relu'.</li>
          <li><strong>solver</strong> método de selección de los pesos: 'lbfgs', 
              'sgd', 'adam'.</li>
          <li><strong>alpha</strong> coeficiente de penalidad de regularización.</li>
          <li><strong>max_iter</strong> número máximo de iteraciones.</li>
          <li><strong>learning_rate</strong> tipo de tasa de aprendizaje utilizado en 
              el gradiente descendente: 'constant', 'invscaling', adaptive'.</li>
          <li><strong>learning_rate_init</strong> tasa de aprendizaje inicial utilizada.</li>
          <li><strong>n_iter_no_change</strong> número máximo de iteraciones que no 
              presenten una mejora determinada.</li>
        </ul>
        Los parámetros en este caso se seleccionan para simular 
        la misma situación analizada con gradiente descendente
        previamente.
    </li>
</ol>

In [ ]:
clf = MLPClassifier(solver='sgd', activation = 'logistic', alpha=1e-5, \
                    hidden_layer_sizes=(2,), max_iter = 1000,\
                    learning_rate = 'constant', learning_rate_init = 0.5,\
                    n_iter_no_change = 600)


<ol start="3">
    <li>Los arreglos de entrada y salida deben ser <strong>NumPy arrays</strong>.
        <ul>
            <li>El arreglo de <strong>X</strong> debe tener <strong>2 dimensiones
                </strong>: cada columna debe corresponder con una variable de 
                entrada y cada fila con una observación particular.</li>
            <li>El arreglo de <strong>Y</strong> debe tener <strong>1 dimensión
                </strong>, siendo cada elemento la salida de cada una de las 
                observaciones</li>
        </ul>
    </li>
</ol>

In [ ]:
X_noBias.shape

In [ ]:
np.ravel(Y)

<ol start="4">
    <li>Se entrena el modelo, lo cual determina los valores de los coeficientes que corresponden al mejor valor de la función costo
    </li>
</ol>

In [ ]:
clf.fit(X_noBias, np.ravel(Y))

<ol start="5">
    <li>Se pueden obtener los atributos del modelo:
        <ul>
            <li>Los valores de las matrices $\Theta$
                del modelo entrenado: el elemento i de 
                la lista representa la matriz de pesos 
                correspondiente a la capa i.
            </li>
        </ul>
    </li>
</ol>

In [ ]:
clf.intercepts_

<ol style="list-style-type:none;">
    <li>
        <ul>
            <li>Los valores de los vectores bias del modelo entrenado: el elemento i de la lista representa el vetor bias correspondiente a la capa i+1.
            </li>
        </ul>
    </li>
</ol>

In [ ]:
clf.coefs_

<ol style="list-style-type:none;">
    <li>
        <ul>
            <li>Los valores de la función costo: el 
                elemento i de la lista representa el 
                valor de $J(\Theta)$ en la iteración i.
            </li>
        </ul>
    </li>
</ol>

In [ ]:
fig2, ax2 = plt.subplots(figsize=(8,4), tight_layout = True)
ax2.plot(clf.loss_curve_);

<ol start="6">
    <li>Se calculan los valores predichos de salida haciendo uso del modelo entrenado.
    </li>
</ol>

In [ ]:
y_skl = clf.predict(X_noBias)

<ul>
    <li>Utilizando los valores de $\Theta$ obtenidos con sklearn, se calculan 
    los valores de salida haciendo uso de la propagación hacia adelante para 
    verificar su correcta implementación.</li>
</ul>

In [ ]:
t1 = np.vstack(((clf.intercepts_[0]), clf.coefs_[0]))
t2 = np.vstack((clf.intercepts_[1], clf.coefs_[1]))
h = predictor(t1, t2)
j = costFunction(X, Y, t1, t2)
j

In [ ]:
skl = widgets.Output()

with skl:
    fig_p, ax_p = plt.subplots(figsize = (8, 4), tight_layout = True)
    fig_p.suptitle(r'Hipótesis vs Valores de salida reales')
    
    ax_p.plot(np.arange(0, len(Y)), y_skl, 'b+', markersize = 8, label = 'Predicción con sklearn', alpha = 0.9)
    
    ax_p.scatter(np.arange(0, len(Y)), Y, marker = 'o', label = 'Salida real', alpha = 0.9, facecolors = 'none', edgecolors = 'g')
    
    lineh_skl, = ax_p.plot(np.arange(0, len(Y)), h, 'rx', markersize = 8, label = 'Predicción', alpha = 0.9)
    
    ax_p.grid(True)
    ax_p.legend(loc="right");

widgets.VBox([skl],layout = layout) 